<a href="https://colab.research.google.com/github/alemando/redesNeuronales/blob/main/notebooks/10-funcion_epsilon_insensitiva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Función epsilon insensitiva
===

Modifique el siguiente código para encontrar los parámetros del modelo usando la función $\epsilon$-insensitiva:

$$
L_\epsilon = 
\begin{cases}
0, & \text{Si } |y - f(x)| \le \epsilon \\
|y - f(x)| - \epsilon, & \text{de lo contrario}
\end{cases}
$$

In [1]:
#
# A continuación se presenta la implementación de un modelo de
# regresión lineal que usa la función de penalización ElasticNet
# para estimar los parámetros óptimos. Complete el código presentado
# para que pasen las pruebas definidas en las celdas restantes.
#
import numpy as np
import pandas as pd
import pytest


class EpsilonRegression:
    def __init__(self, intercept, coef, maxiter, mu, epsilon):
        self.intercept_ = intercept
        self.coef_ = np.array(coef)
        self._maxiter = maxiter
        self._mu = mu
        self._epsilon = epsilon

    def compute_loss(self, x, y):
        d = self.predict(x)
        error = np.array([abs(yi - di) for yi, di in zip(y, d)])
        loss = sum([e  - self._epsilon if e > self._epsilon else 0 for e in error])
        return loss

    def predict(self, x):
        return [self.coef_[0]*d[0] + self.coef_[1]*d[1] + self.intercept_ for d in x]

    def compute_gradient(self, x, y):
        grd = []
        delta = 0.0001
        W = [self.coef_[0], self.coef_[1], self.intercept_]
        L0 = self.compute_loss(x,y)
        for i in range(len(W)):
          W[i] += delta
          deltaD = [W[0]*i[0] + W[1]*i[1] + W[2] for i in x]
          errorD = [abs(yi - di) for yi, di in zip(y, deltaD)]
          L = sum([e  - self._epsilon if e > self._epsilon else 0 for e in errorD])
          grd.append((L-L0)/delta)
          W[i] -= delta
        self._grad_coef = np.array([grd[0], grd[1]])
        self._grad_intercept = grd[2]

    def fit(self, x, y):
        for iter in range(self._maxiter):
            self.compute_gradient(x, y)
            self.improve()

    def improve(self):
        self.intercept_ = self.intercept_ - self._mu * self._grad_intercept
        self.coef_ = self.coef_ - self._mu * self._grad_coef


x = [
    [0.0, 0.1],
    [0.2, 0.3],
    [0.4, 0.5],
    [0.6, 0.7],
    [0.8, 0.9],
    [1.0, 1.1],
]

# y = 1 x1 + 1.1 x2 + 0.2
y = [
    0.31,
    0.73,
    1.15,
    1.57,
    1.99,
    2.41,
]

In [2]:
#
# Test 1
# =============================================================================
# Implemente la función de pérdida.
#
# Rta/
# True
#

# ---->>> Evaluación ---->>>
lr = EpsilonRegression(
    intercept=0.1,
    coef=[0.2, 0.3],
    maxiter=10000,
    mu=0.001,
    epsilon=0.1,
)
pytest.approx(lr.compute_loss(x, y), 0.0001) == 5.28

True

In [3]:
#
# Test 2
# =============================================================================
# Implemente la función de pronóstico
#
# Rta/
# True
#

# ---->>> Evaluación ---->>>
lr = EpsilonRegression(
    intercept=0.1,
    coef=[0.2, 0.3],
    maxiter=10000,
    mu=0.001,
    epsilon=0.1,
)
all(
    pytest.approx(a) == b
    for a, b in zip(lr.predict(x), [0.13, 0.23, 0.33, 0.43, 0.53, 0.63])
)

True

In [4]:
#
# Test 3
# =============================================================================
# Implemente el gradiente
#
# Rta/
# True
# True
#

# ---->>> Evaluación ---->>>
lr = EpsilonRegression(
    intercept=0.1,
    coef=[0.2, 0.3],
    maxiter=10000,
    mu=0.001,
    epsilon=0.1,
)
lr.compute_gradient(x, y)
print(lr._grad_intercept == pytest.approx(-6.0))
print(all(pytest.approx(a) == b for a, b in zip(lr._grad_coef, [-3.0 , -3.6])))

True
True


In [5]:
#
# Test 4
# =============================================================================
# Implemente la función fit
#
# Rta/
# True
# True
#

# ---->>> Evaluación ---->>>
lr = EpsilonRegression(
    intercept=0.1,
    coef=[0.2, 0.3],
    maxiter=1000,
    mu=0.001,
    epsilon=0.1,
)
lr.fit(x, y)
print(pytest.approx(lr.intercept_, 0.001) == 0.3080)
print(all(pytest.approx(a, 0.001) == b for a, b in zip(lr.coef_, [0.8902, 1.011])))

True
True
